In [1]:
%matplotlib inline

Семинар 1: Knowledge Distillation
===============================

**Материал взят из [Источника](https://github.com/AlexandrosChrtn)**


Knowledge distillation - это метод, который позволяет передавать знания от больших, требующих больших вычислительных затрат моделей к более мелким без существенной потери точности. Это позволяет использовать менее мощное аппаратное обеспечение, что ускоряет и повышает эффективность инференса.

В этом руководстве мы проведем ряд экспериментов, направленных на повышение точности работы маленькой нейронной сети **(студента)**, дистиллируя знания из большой сети **(учителя)**. Вычислительные затраты и скорость маленькой сети останутся неизменными, наше вмешательство фокусируется только на весах модели, а не на forward pass (прямом проходе). Knowledge distillation находит применение в таких устройствах, как дроны и мобильные телефоны.
 
 
В этом ноутбуке мы узнаем:

- Как модифицировать классы моделей для извлечения скрытых представлений и использования их для дальнейших вычислений

- Как повысить качество маленькой модели за счет использования более сложной модели в качестве учителей

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from tqdm import tqdm
import numpy as np
import random

def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Check if GPU is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CIFAR-10
================

В качестве данных мы будем использовать **CIFAR-10**, один из самых известных датасетов для классификации изображенией

![Example of CIFAR-10
images](https://pytorch.org/tutorials//../_static/img/cifar10.png)


In [3]:
# Below we are preprocessing data for CIFAR-10. We use an arbitrary batch size of 128.
transforms_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ImageNet normalize
])

# Loading the CIFAR-10 dataset:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms_cifar)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_cifar)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
#Dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

Определение классов моделей и полезные функции
============================================

Далее нам нужно закодить классы реализующие модели классификации. Мы зададим две сверточные архитектуры (CNNs) с различным количеством сверточных слоев, они служат для извлечения признаков, и с полносвязными классификаторами с 10 классами. Для модели студента количество фильтров и нейронов меньше, чем для модели учителя.

In [5]:
# Deeper neural network class to be used as teacher:
class DeepNN(nn.Module):
    def __init__(self, num_classes=10):
        super(DeepNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
# Lightweight neural network class to be used as student:
class LightNN(nn.Module):
    def __init__(self, num_classes=10):
        super(LightNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

![Train both networks with Cross-Entropy. The student will be used as a
baseline:](https://pytorch.org/tutorials//../_static/img/knowledge_distillation/ce_only.png)

Обучение и тест
===============

Мы используем 2 функции, которые помогают нам получать и оценивать результаты в нашей
первоначальной задаче классификации.



In [7]:
def train(model, train_loader, epochs, learning_rate, device):
    """
    `model`: A model instance to train (update its weights) via this
    function.
    `train_loader`: We defined our `train_loader` above, and its job is
    to feed the data into the model.
    `epochs`: How many times we loop over the dataset.
    `learning_rate`: The learning rate determines how large our steps
    towards convergence should be. Too large or too small steps can be
    detrimental.
    `device`: Determines the device to run the workload on. Can be
    either CPU or GPU depending on availability.
    """
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs: A collection of batch_size images
            # labels: A vector of dimensionality batch_size with integers denoting class of each image
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # outputs: Output of the network for the collection of images. A tensor of dimensionality batch_size x num_classes
            # labels: The actual labels of the images. Vector of dimensionality batch_size
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

def test(model, test_loader, device):
    """
    `model`: A model instance to train (update its weights) via this
    function.
    `test_loader`: We defined our `test_loader` above, and its job is
    to feed the data into the model.
    `device`: Determines the device to run the workload on. Can be
    either CPU or GPU depending on availability.
    """
    
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

Cross-entropy (baseline)
==================


In [8]:
# Instantiate the teacher network:

set_global_seed(42)
nn_deep = DeepNN(num_classes=10).to(device)
train(nn_deep, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_deep = test(nn_deep, test_loader, device)

Epoch 1/10, Loss: 1.3256008115875753
Epoch 2/10, Loss: 0.8632667924437072
Epoch 3/10, Loss: 0.6770264405728607
Epoch 4/10, Loss: 0.5343857654525191
Epoch 5/10, Loss: 0.41441617887038407
Epoch 6/10, Loss: 0.3134289937822715
Epoch 7/10, Loss: 0.22761023200838768
Epoch 8/10, Loss: 0.17139672357446092
Epoch 9/10, Loss: 0.14778140834187303
Epoch 10/10, Loss: 0.1222293549896125
Test Accuracy: 74.23%


In [9]:
# Instantiate the student network:

set_global_seed(42)
nn_light = LightNN(num_classes=10).to(device)

train(nn_light, train_loader, epochs=10, learning_rate=0.001, device=device)
test_accuracy_light_ce = test(nn_light, test_loader, device)

Epoch 1/10, Loss: 1.4668227635381166
Epoch 2/10, Loss: 1.1526997007067552
Epoch 3/10, Loss: 1.0217017305781468
Epoch 4/10, Loss: 0.9182852922802995
Epoch 5/10, Loss: 0.844304761465858
Epoch 6/10, Loss: 0.7765299892791396
Epoch 7/10, Loss: 0.7096788134721234
Epoch 8/10, Loss: 0.649937776348475
Epoch 9/10, Loss: 0.5978364141853264
Epoch 10/10, Loss: 0.5463351162955584
Test Accuracy: 70.23%


При сравнении результатов нам важно знать число параметров модели


In [10]:
total_params_deep = "{:,}".format(sum(p.numel() for p in nn_deep.parameters()))
print(f"DeepNN parameters: {total_params_deep}")
total_params_light = "{:,}".format(sum(p.numel() for p in nn_light.parameters()))
print(f"LightNN parameters: {total_params_light}")

DeepNN parameters: 1,186,986
LightNN parameters: 267,738


Без использования Knowledge distillation мы получим следующую точность на тесте, то есть наш baseline

In [11]:
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy: {test_accuracy_light_ce:.2f}%")

Teacher accuracy: 74.23%
Student accuracy: 70.23%


Knowledge distillation run
==========================


Теперь давайте попробуем повысить точность на тесте для сети студента, дистиллировав в нее учителя. Knowledge distillation - простой метод для достижения этой цели, основанный на том факте, что обе сети выводят распределение вероятностей по нашим классам. Следовательно, обе сети используют одинаковое количество выходных нейронов. Этот метод работает за счет добавления дополнительного лосса к традиционому Cross-Entropy loss. Предполагается, что выход сети учителя содержит дополнительную информацию, которая может быть использована сетью студента во время обучения.

Например, в CIFAR-10 грузовик можно принять за автомобиль или самолет, если у него есть колеса, но маловероятно, что его примут за собаку. Следовательно, имеет смысл предположить, что полезная информация содержится не только в вероятном прогнозе
правильно обученного учителя, но и во всем распределении выходных данных. Однако сам по себе Cross-Entropy loss недостаточно использует эту информацию, поскольку активации для маловероятных классов, как правило, настолько малы, что градиенты не приводят к значительному изменению весов.

![Distillation loss is calculated from the logits of the networks. It
only returns gradients to the
student:](https://pytorch.org/tutorials//../_static/img/knowledge_distillation/distillation_output_loss.png)


In [12]:
def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight, device):
    
    """
    `T`: Temperature controls the smoothness of the output
    distributions. Larger `T` leads to smoother distributions, thus
    smaller probabilities get a larger boost.
    `soft_target_loss_weight`: A weight assigned to the extra objective
    we\'re about to include.
    `ce_loss_weight`: A weight assigned to cross-entropy. Tuning these
    weights pushes the network towards optimizing for either objective.
    """
    
    ce_loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = torch.sum(soft_targets * (soft_targets.log() - soft_prob)) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")


In [13]:
set_global_seed(42)
new_nn_light = LightNN(num_classes=10).to(device)

# Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
train_knowledge_distillation(teacher=nn_deep, student=new_nn_light, train_loader=train_loader, epochs=10, learning_rate=0.001, T=2, soft_target_loss_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_kd = test(new_nn_light, test_loader, device)

# Compare the student test accuracy with and without the teacher, after distillation
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")

Epoch 1/10, Loss: 2.403964203946731
Epoch 2/10, Loss: 1.8924736418687473
Epoch 3/10, Loss: 1.6759761016996926
Epoch 4/10, Loss: 1.5063083873075598
Epoch 5/10, Loss: 1.3776447824809863
Epoch 6/10, Loss: 1.2682764351825275
Epoch 7/10, Loss: 1.1630251947266366
Epoch 8/10, Loss: 1.0745595810968247
Epoch 9/10, Loss: 0.9920528576806988
Epoch 10/10, Loss: 0.9227680528865141
Test Accuracy: 70.34%
Teacher accuracy: 74.23%
Student accuracy without teacher: 70.23%
Student accuracy with CE + KD: 70.34%


Не стесняйтесь экспериментировать с параметром температуры, который управляет гладкостью функции softmax и коэффициентами в функции потерь. В нейронных сетях легко включить дополнительные функции потерь в основные задачи для улучшения обобщающей способности. Давайте попробуем включить оптимизируемый функционал для студента, но теперь давайте сосредоточимся на его скрытых состояниях, а не на выходных слоях. Наша цель - донести информацию из представления учителя до студента, включив наивную функцию потерь, минимизация которой подразумевает, что сглаженные векторы, которые впоследствии передаются в классификаторы, становятся более похожими по мере уменьшения потерь.

Конечно, учитель не обновляет свои веса, поэтому минимизация зависит только от веса ученика. Логическим обоснованием этого метода является то, что мы исходим из предположения, что модель учителя имеет лучшее внутреннее представление, которое вряд ли может быть достигнуто студентом без внешнего вмешательства, поэтому мы искусственно подталкиваем студента к имитации внутреннего представления учителя. Однако не ясно, поможет ли это в конечном итоге студенту, потому что использование облегченной сети для достижения этой цели может быть полезным, если предположить, что мы нашли внутреннее представление, которое повышает точность на тесте, но это также может быть вредным, поскольку сети имеют разную архитектуру и сложность. Студент не обладает такой же способностью к обучению, как учитель. Другими словами, нет никаких причин для того, чтобы эти два вектора (вектор студента и вектор учителя) совпадали по каждому компоненту. Ученик мог бы создать внутреннее представление, которое является заменой представления учителя, и это было бы столь же эффективно. Тем не менее, мы все еще можем провести небольшой эксперимент, чтобы выяснить эффективность этого метода. Мы будем использовать значение `CosineEmbeddingLoss`, которое задается следующей формулой:


![Formula for
CosineEmbeddingLoss](https://pytorch.org/tutorials//../_static/img/knowledge_distillation/cosine_embedding_loss.png)


Очевидно, что сначала нам нужно решить одну проблему. Когда мы
применяли дистилляцию к выходному слою, мы упомянули, что обе сети
имеют одинаковое количество нейронов, равное количеству классов.
Однако это не относится к слою, следующему за нашими сверточными
слоями. Здесь у учителя больше нейронов, чем у ученика, после
выравнивания последнего сверточного слоя. Наша функция потерь принимает
в качестве входных данных два вектора одинаковой размерности, поэтому нам нужно
каким-то образом сопоставить их. Мы решим эту проблему, включив avgPoll, следующий за сверточным слоем учителя, чтобы уменьшить его
размерность в соответствии с размером студента.

Чтобы продолжить, мы немного изменим классы моделей. Теперь
функция forward возвращает не только логиты сети, но и
сглаженное скрытое представление после сверточного слоя. Мы
добавили вышеупомянутое изменение:

In [14]:
class ModifiedDeepNNCosine(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedDeepNNCosine, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        flattened_conv_output = torch.flatten(x, 1)
        x = self.classifier(flattened_conv_output)
        flattened_conv_output_after_pooling = torch.nn.functional.avg_pool1d(flattened_conv_output, 2)
        return x, flattened_conv_output_after_pooling

# Create a similar student class where we return a tuple. We do not apply pooling after flattening.
class ModifiedLightNNCosine(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedLightNNCosine, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        flattened_conv_output = torch.flatten(x, 1)
        x = self.classifier(flattened_conv_output)
        return x, flattened_conv_output

In [15]:
# We do not have to train the modified deep network from scratch of course, we just load its weights from the trained instance
modified_nn_deep = ModifiedDeepNNCosine(num_classes=10).to(device)
modified_nn_deep.load_state_dict(nn_deep.state_dict())

# Once again ensure the norm of the first layer is the same for both networks
print("Norm of 1st layer for deep_nn:", torch.norm(nn_deep.features[0].weight).item())
print("Norm of 1st layer for modified_deep_nn:", torch.norm(modified_nn_deep.features[0].weight).item())

# Initialize a modified lightweight network with the same seed as our other lightweight instances. This will be trained from scratch to examine the effectiveness of cosine loss minimization.
set_global_seed(42)
modified_nn_light = ModifiedLightNNCosine(num_classes=10).to(device)
print("Norm of 1st layer:", torch.norm(modified_nn_light.features[0].weight).item())

Norm of 1st layer for deep_nn: 7.4885735511779785
Norm of 1st layer for modified_deep_nn: 7.4885735511779785
Norm of 1st layer: 2.327361822128296


Естественно, нам нужно изменить цикл train, потому что теперь модель
возвращает кортеж `(logits, hidden_representation)`. Используя
тензор входных данных, мы можем вывести их формы.


In [16]:
# Create a sample input tensor
sample_input = torch.randn(128, 3, 32, 32).to(device) # Batch size: 128, Filters: 3, Image size: 32x32

# Pass the input through the student
logits, hidden_representation = modified_nn_light(sample_input)

# Print the shapes of the tensors
print("Student logits shape:", logits.shape) # batch_size x total_classes
print("Student hidden representation shape:", hidden_representation.shape) # batch_size x hidden_representation_size

# Pass the input through the teacher
logits, hidden_representation = modified_nn_deep(sample_input)

# Print the shapes of the tensors
print("Teacher logits shape:", logits.shape) # batch_size x total_classes
print("Teacher hidden representation shape:", hidden_representation.shape) # batch_size x hidden_representation_size

Student logits shape: torch.Size([128, 10])
Student hidden representation shape: torch.Size([128, 1024])
Teacher logits shape: torch.Size([128, 10])
Teacher hidden representation shape: torch.Size([128, 1024])


![In Cosine Loss minimization, we want to maximize the cosine similarity
of the two representations by returning gradients to the
student:](https://pytorch.org/tutorials//../_static/img/knowledge_distillation/cosine_loss_distillation.png)

В нашем случае `hidden_representation_size` равно `1024`. Это карта фичей последнего сверточного слоя студента, и, как вы можете видеть, это входные данные для его классификатора. Для учителя это тоже `1024`, потому что мы сделали это с помощью `avg_pool1d` из `2048`.
Потери в карте фичей, применяемые здесь, влияют только на вес студента до
расчета основной функции потерь. Другими словами, это не влияет на классификатор
студента. Модифицированный цикл обучения выглядит следующим образом:


In [17]:
def train_cosine_loss(teacher, student, train_loader, epochs, learning_rate, hidden_rep_loss_weight, ce_loss_weight, device):
    ce_loss = nn.CrossEntropyLoss()
    cosine_loss = nn.CosineEmbeddingLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.to(device)
    student.to(device)
    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # Forward pass with the teacher model and keep only the hidden representation
            with torch.no_grad():
                _, teacher_hidden_representation = teacher(inputs)

            # Forward pass with the student model
            student_logits, student_hidden_representation = student(inputs)

            # Calculate the cosine loss. Target is a vector of ones. From the loss formula above we can see that is the case where loss minimization leads to cosine similarity increase.
            hidden_rep_loss = cosine_loss(student_hidden_representation, teacher_hidden_representation, target=torch.ones(inputs.size(0)).to(device))

            # Calculate the true label loss
            label_loss = ce_loss(student_logits, labels)

            # Weighted sum of the two losses
            loss = hidden_rep_loss_weight * hidden_rep_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

По той же причине нам нужно изменить нашу тестовую функцию. Здесь мы игнорируем
скрытое представление, возвращаемое моделью.

In [18]:
def test_multiple_outputs(model, test_loader, device):
    model.to(device)
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs, _ = model(inputs) # Disregard the second tensor of the tuple
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

В этом случае мы могли бы легко включить
в одну и ту же функцию как извлечение знаний, так и минимизацию косинусных потерь. Обычно для
достижения более высокой производительности в парадигмах учитель-ученик используются комбинированные методы. На
данный момент мы можем обучить модели.

In [19]:
# Train and test the lightweight network with cross entropy loss
train_cosine_loss(teacher=modified_nn_deep, student=modified_nn_light, train_loader=train_loader, epochs=10, learning_rate=0.001, hidden_rep_loss_weight=0.25, ce_loss_weight=0.75, device=device)
test_accuracy_light_ce_and_cosine_loss = test_multiple_outputs(modified_nn_light, test_loader, device)

Epoch 1/10, Loss: 1.305133652809026
Epoch 2/10, Loss: 1.071263348964779
Epoch 3/10, Loss: 0.9721636122754772
Epoch 4/10, Loss: 0.8992896465694203
Epoch 5/10, Loss: 0.8461023011170995
Epoch 6/10, Loss: 0.7991485271002631
Epoch 7/10, Loss: 0.7561488685095706
Epoch 8/10, Loss: 0.71864186757056
Epoch 9/10, Loss: 0.6837123874813089
Epoch 10/10, Loss: 0.655079073308374
Test Accuracy: 70.93%


In [20]:
print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")
print(f"Student accuracy with CE + CosineLoss: {test_accuracy_light_ce_and_cosine_loss:.2f}%")

Teacher accuracy: 74.23%
Student accuracy without teacher: 70.23%
Student accuracy with CE + KD: 70.34%
Student accuracy with CE + CosineLoss: 70.93%


Заключение
==========

Ни один из вышеперечисленных методов не увеличивает количество параметров
сети или время вывода, поэтому увеличение производительности достигается за счет
небольших затрат на вычисление градиентов во время обучения. В
приложениях ML мы в основном заботимся о времени вывода, поскольку обучение
происходит до развертывания модели. Если наша упрощенная модель все еще
слишком сложна для внедрения, мы можем применить другие идеи, такие как
квантование после обучения **об этом в следующих лекциях**. Идеи Knowledge Distillation могут быть применены во многих задачах, а не только в классификации, и вы можете поэкспериментировать с величинами
например, коэффициенты, температура или количество нейронов. Не стесняйтесь изменять
любые цифры из ноутбука, но имейте в виду, что при изменении
количества нейронов / фильтров может возникнуть несоответствие архитектур.

Основные статьи:

-   [Hinton, G., Vinyals, O., Dean, J.: Distilling the knowledge in a
    neural network. In: Neural Information Processing System Deep
    Learning Workshop (2015)](https://arxiv.org/abs/1503.02531)
-   [Romero, A., Ballas, N., Kahou, S.E., Chassang, A., Gatta, C.,
    Bengio, Y.: Fitnets: Hints for thin deep nets. In: Proceedings of
    the International Conference on Learning
    Representations (2015)](https://arxiv.org/abs/1412.6550)
